In [7]:

#___________________________________________________________________________________________________________________________________________________________

#                                                       LIIBRERIAS NECESARIAS

import os
import csv
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
import pandas as pd
from sqlalchemy import create_engine
import urllib.parse  # Agregado para construir la cadena de conexión
import shutil
import tempfile
from datetime import datetime  # Agregamos la importación de datetime


#___________________________________________________________________________________________________________________________________________________________

#                                                 RELLENO DE NULOS CARGA Y ACTUALIZACION ARCHIVOS GOOGLE DRIVE

# Directorio donde se encuentra el archivo JSON de credenciales
directorio_credenciales = "data-424019-28bfddebf741.json"

# Cargar credenciales desde el archivo JSON descargado
credentials = service_account.Credentials.from_service_account_file(directorio_credenciales)

# Crear cliente para acceder a la API de Google Drive
drive_service = build('drive', 'v3', credentials=credentials)

# ID de la carpeta en Google Drive
folder_id = '1tZHS0BuJoSG3DsfSmxgY5Am3Llj24JPY'

# Función para descargar un archivo de Google Drive
def descargar_archivo(file_id, download_path):
    request = drive_service.files().get_media(fileId=file_id)
    with open(download_path, 'wb') as fh:
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"Descargado {int(status.progress() * 100)}%.")

# Función para subir un archivo a Google Drive
def subir_archivo(file_id, upload_file_path):
    media = MediaFileUpload(upload_file_path, resumable=True)
    updated_file = drive_service.files().update(fileId=file_id, media_body=media).execute()
    print('Archivo actualizado en Google Drive:', updated_file.get('name'))

# Directorio temporal para manipular archivos
temp_dir = tempfile.gettempdir()

# Lista para almacenar los archivos modificados
archivos_modificados = []

# Obtener lista de archivos en la carpeta de Google Drive
results = drive_service.files().list(q=f"'{folder_id}' in parents", fields='files(id, name)').execute()
files = results.get('files', [])

# Iterar sobre los archivos
for file in files:
    file_id = file['id']
    file_name = file['name']
    download_path = os.path.join(temp_dir, file_name)

    # Descargar el archivo de Google Drive
    descargar_archivo(file_id, download_path)

    # Leer el archivo CSV y hacer modificaciones
    with open(download_path, newline='') as csvfile:
        reader = csv.reader(csvfile)
        rows = list(reader)
        for row_index, row in enumerate(rows):
            for col_index, cell in enumerate(row):
                if not cell:
                    rows[row_index][col_index] = "no definido"

    # Guardar los cambios en el archivo temporal sin la extensión _temp.csv
    download_path_temp = os.path.join(temp_dir, os.path.splitext(file_name)[0])
    with open(download_path_temp, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(rows)

    archivos_modificados.append((file_id, download_path_temp))

# Subir los archivos modificados a Google Drive
for file_id, download_path_temp in archivos_modificados:
    subir_archivo(file_id, download_path_temp)

# Guardar la marca de tiempo de la última ejecución
# Puedes cambiar esto según cómo desees formatear la marca de tiempo
marca_de_tiempo_actual = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
with open("ultima_ejecucion.txt", "w") as file:
    file.write(marca_de_tiempo_actual)


#________________________________________________________________________________________________________________________________________________

#                           VERIFICACION CUALES SON LAS FILAS NUEVAS PARA LUEGO CARGAR A SQL

# Directorio donde se encuentra el archivo JSON de credenciales
directorio_credenciales = "data-424019-28bfddebf741.json"

# Cargar credenciales desde el archivo JSON descargado
credentials = service_account.Credentials.from_service_account_file(directorio_credenciales)

# Crear cliente para acceder a la API de Google Drive
drive_service = build('drive', 'v3', credentials=credentials)

# IDs de las carpetas en Google Drive
source_folder_id = '1tZHS0BuJoSG3DsfSmxgY5Am3Llj24JPY'  # ID de la carpeta de destino
comparison_folder_id = '1ZMAqOISfz-z5kv_4OgGZT5lI1C9S3Xwb'  # ID de la carpeta de comparación
destination_folder_id = '18gXDMFkbWoGDXfRN6JVcmQdi791cHZ9G'  # ID de la carpeta de origen



# Función para descargar un archivo de Google Drive
def descargar_archivo(file_id, download_path, folder_name):
    request = drive_service.files().get_media(fileId=file_id)
    with open(download_path, 'wb') as fh:
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"Descargando archivo '{folder_name}/{os.path.basename(download_path)}' desde la carpeta '{folder_name}'.")

# Función para subir un archivo a una carpeta específica en Google Drive
def subir_archivo_a_carpeta(upload_file_path, folder_id, file_name):
    media = MediaFileUpload(upload_file_path, resumable=True)
    file_metadata = {
        'name': file_name,
        'parents': [folder_id]
    }
    drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    print(f'Archivo {file_name} subido a Google Drive en la carpeta especificada: {folder_id}')

# Directorio temporal para manipular archivos
temp_dir = tempfile.gettempdir()

# Obtener lista de archivos en la carpeta de origen de Google Drive
results = drive_service.files().list(q=f"'{source_folder_id}' in parents", fields='files(id, name)').execute()
source_files = results.get('files', [])

# Leer los archivos de la carpeta de origen y almacenarlos en un diccionario
origen_data = {}
for file in source_files:
    file_id = file['id']
    file_name = file['name']
    download_path_origen = os.path.join(temp_dir, file_name)
    
    # Descargar archivo de la carpeta de origen
    descargar_archivo(file_id, download_path_origen, 'Carpeta de origen')
    
    # Leer el archivo CSV y almacenar los datos en el diccionario
    origen_data[file_name] = pd.read_csv(download_path_origen)
    print(f"Datos del archivo de origen '{file_name}':")

# Obtener lista de archivos en la carpeta de comparación de Google Drive
results_comparison = drive_service.files().list(q=f"'{comparison_folder_id}' in parents", fields='files(id, name)').execute()
comparison_files_list = results_comparison.get('files', [])

# Definir comparison_files como un diccionario vacío para almacenar los nombres de archivo y sus datos
comparison_files = {}

# Leer los archivos de la carpeta de comparación y almacenarlos en un diccionario
for file in comparison_files_list:
    file_id = file['id']
    file_name = file['name']
    download_path_comparacion = os.path.join(temp_dir, f"comparison_{file_name}")
    
    # Descargar archivo de la carpeta de comparación
    descargar_archivo(file_id, download_path_comparacion, 'Carpeta de comparación')
    
    # Leer el archivo CSV y almacenar los datos en el diccionario
    comparison_files[file_name] = pd.read_csv(download_path_comparacion)
    print(f"Datos del archivo de comparación '{file_name}':")
    
# Realizar la comparación entre los datos de origen y los datos de comparación
for file_name, df_comparacion in comparison_files.items():
    if file_name in origen_data:
        print(f"Realizando comparación entre los datos de '{file_name}' en la carpeta de origen y en la carpeta de comparación.")
        
        df_origen = origen_data[file_name]
        
        # Verificar y convertir las columnas para que tengan el mismo tipo de datos
        columnas_comunes = list(set(df_origen.columns) & set(df_comparacion.columns))
        for columna in columnas_comunes:
            tipo_origen = df_origen[columna].dtype
            tipo_comparacion = df_comparacion[columna].dtype
            if tipo_origen != tipo_comparacion:
                if tipo_origen == 'object' or tipo_comparacion == 'object':
                    # Convertir a tipo de datos string si alguna de las columnas es de tipo object
                    df_origen[columna] = df_origen[columna].astype(str)
                    df_comparacion[columna] = df_comparacion[columna].astype(str)
                else:
                    # Convertir ambas columnas al tipo de datos más general (por ejemplo, float64)
                    tipo_general = max(tipo_origen, tipo_comparacion)
                    df_origen[columna] = df_origen[columna].astype(tipo_general)
                    df_comparacion[columna] = df_comparacion[columna].astype(tipo_general)

        # Comparar y obtener las filas nuevas (las filas que están en df_origen pero no en df_comparacion)
        df_nuevas_filas = df_origen.merge(df_comparacion, indicator=True, how='left').loc[lambda x: x['_merge'] == 'left_only'].drop(columns=['_merge'])
        if not df_nuevas_filas.empty:
            print(f"Se encontraron filas nuevas en el archivo '{file_name}':")
            

                # Función para subir un archivo a una carpeta específica en Google Drive
            def subir_archivo_a_carpeta(upload_file_path, folder_id, file_name):
                # Borrar todos los archivos existentes en la carpeta de destino
                files = drive_service.files().list(q=f"'{folder_id}' in parents", fields='files(id)').execute()
                for file in files.get('files', []):
                    drive_service.files().delete(fileId=file.get('id')).execute()
                    print(f"Archivo '{file_name}' eliminado de la carpeta de destino.")

                # Subir el archivo especificado a la carpeta de destino
                media = MediaFileUpload(upload_file_path, resumable=True)
                file_metadata = {
                    'name': file_name,
                    'parents': [folder_id]
                }
                drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
                print(f'Archivo {file_name} subido a Google Drive en la carpeta especificada: {folder_id}')

            # Subir el DataFrame con las filas nuevas a la carpeta de destino
            nuevo_nombre_path = os.path.join(temp_dir, file_name)
            with open(nuevo_nombre_path, 'w', newline='') as file:
                df_nuevas_filas.to_csv(file, index=False)
            subir_archivo_a_carpeta(nuevo_nombre_path, destination_folder_id, file_name)
            print(f"El archivo '{file_name}' con las filas nuevas se ha subido correctamente a la carpeta de destino.")
        else:
            print(f"No se encontraron filas nuevas en el archivo '{file_name}'.")
    else:
        print(f"No se encontró el archivo '{file_name}' en la carpeta de origen.")

#___________________________________________________________________________________________________________________________________________________________

#                                               CARGA DE DATOS SQL

# Definir la ruta al archivo JSON de credenciales de Google Drive
credentials_file = 'data-424019-28bfddebf741.json'

# Cargar las credenciales desde el archivo JSON
credentials = service_account.Credentials.from_service_account_file(credentials_file)

# Crear un cliente para acceder a la API de Google Drive
drive_service = build('drive', 'v3', credentials=credentials)

# ID de la carpeta en Google Drive donde se encuentran los archivos CSV
folder_id = '18gXDMFkbWoGDXfRN6JVcmQdi791cHZ9G'

# Directorio base donde se guardarán los archivos CSV descargados
base_directory = '/ruta/a/tu/directorio/base'
csv_folder_name = 'archivos_csv_google_drive'
local_directory = os.path.join(base_directory, csv_folder_name)

# Función para descargar archivos CSV de Google Drive
def download_csv_files_from_drive(folder_id, local_directory):
    # Eliminar archivos CSV antiguos en el directorio local
    for file in os.listdir(local_directory):
        if file.endswith('.csv'):
            os.remove(os.path.join(local_directory, file))
            print(f"Removed old CSV file: {file}")
    
    # Descargar los nuevos archivos CSV desde Google Drive
    results = drive_service.files().list(q=f"'{folder_id}' in parents", fields='files(id, name)').execute()
    files = results.get('files', [])
    for file in files:
        file_id = file['id']
        file_name = file['name']
        download_path = os.path.join(local_directory, file_name)
        request = drive_service.files().get_media(fileId=file_id)
        with open(download_path, 'wb') as fh:
            downloader = MediaIoBaseDownload(fh, request)
            done = False
            while not done:
                status, done = downloader.next_chunk()
                print(f"Downloaded {int(status.progress() * 100)}% of {file_name}")

    # Cambiar el delimitador de todos los archivos CSV en el directorio
    # Obtener la lista de archivos CSV en el directorio
    csv_files = os.listdir(local_directory)

    # Iterar sobre cada archivo CSV y cambiar el delimitador
    for file_name in csv_files:
        if file_name.endswith('.csv'):
            file_path = os.path.join(local_directory, file_name)
            # Leer el archivo CSV con la coma como delimitador original
            df = pd.read_csv(file_path)
            # Cambiar el delimitador de la coma a punto y coma
            df.to_csv(file_path, sep=';', index=False)

    print("Se han cambiado los delimitadores de todos los archivos CSV en el directorio por ';'.")


# Función para cargar datos desde archivos CSV a una base de datos SQL Server
def load_csv_data_to_sql(csv_directory, conn_str):
    files = os.listdir(csv_directory)
    for file in files:
        if file.endswith('.csv'):
            file_path = os.path.join(csv_directory, file)
            table_name = os.path.splitext(file)[0]  # Utilizar el nombre del archivo CSV como nombre de la tabla
            
            # Define el delimitador predeterminado
            delimiter = ';'

                
            # Define una función de redondeo personalizada
            def custom_round(value):
                if isinstance(value, float):
                    return round(value, 2)
                return value
            
            # Leer el archivo CSV con el delimitador correcto y aplicar la función de redondeo personalizada
            df = pd.read_csv(file_path, delimiter=delimiter, converters={i: custom_round for i in range(10)})  # ajusta el rango 10
            
            # Crear el motor de conexión a la base de datos
            engine = create_engine(conn_str)
            
            # Cargar los datos en SQL Server
            df.to_sql(name=table_name, con=engine, if_exists='append', index=False)
            print(f"Data from {file} loaded into SQL table {table_name}")

# Ejemplo de uso
if __name__ == "__main__":
    # Crear el directorio local si no existe
    if not os.path.exists(local_directory):
        os.makedirs(local_directory)

    # Descargar archivos CSV de Google Drive a un directorio local
    download_csv_files_from_drive(folder_id, local_directory)
    
    # Construir la cadena de conexión para SQLAlchemy
    params = urllib.parse.quote_plus("DRIVER={ODBC Driver 17 for SQL Server};SERVER=JULIAN;DATABASE=Top-Drinks;Trusted_Connection=yes;")
    conn_str = f"mssql+pyodbc:///?odbc_connect={params}"
    
    # Cargar datos desde archivos CSV a una base de datos SQL Server
    load_csv_data_to_sql(local_directory, conn_str)

#___________________________________________________________________________________________________________________________________________________________

#                                                 ACTULIZAR LA DATA QUE SE ESTA USANDO PARA LA COMPARACION



# Directorio donde se encuentra el archivo JSON de credenciales
directorio_credenciales = "data-424019-28bfddebf741.json"

# Cargar credenciales desde el archivo JSON descargado
credentials = service_account.Credentials.from_service_account_file(directorio_credenciales)

# Crear cliente para acceder a la API de Google Drive
drive_service = build('drive', 'v3', credentials=credentials)

# ID de la carpeta de origen en Google Drive
source_folder_id = '1tZHS0BuJoSG3DsfSmxgY5Am3Llj24JPY'
# ID de la carpeta de destino en Google Drive
destination_id = '1ZMAqOISfz-z5kv_4OgGZT5lI1C9S3Xwb'  # Reemplaza 'your_destination_folder_id' con el ID de tu carpeta de destino

# Directorio temporal para manipular archivos
temp_dir = tempfile.gettempdir()

# Función para descargar un archivo de Google Drive
def descargar_archivo(file_id, download_path):
    request = drive_service.files().get_media(fileId=file_id)
    with open(download_path, 'wb') as fh:
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"Descargado {int(status.progress() * 100)}%.")

# Función para subir un archivo a una carpeta específica en Google Drive
def subir_archivo_a_carpeta(upload_file_path, file_name):
    # Buscar el archivo en la carpeta de destino por su nombre
    response = drive_service.files().list(
        q=f"name='{file_name}' and '{destination_id}' in parents",
        fields='files(id)'
    ).execute()
    
    # Si se encuentra el archivo, obtener su ID y actualizarlo
    if response.get('files', []):
        file_id = response['files'][0]['id']
        media = MediaFileUpload(upload_file_path)
        drive_service.files().update(fileId=file_id, media_body=media).execute()
        print(f'Archivo actualizado en Google Drive: {file_name}')
    else:
        # Si el archivo no existe, crear uno nuevo en la carpeta de destino
        media = MediaFileUpload(upload_file_path, resumable=True)
        file_metadata = {
            'name': file_name,
            'parents': [destination_id]
        }
        drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
        print(f'Archivo subido a Google Drive en la carpeta especificada: {destination_id}')

# Obtener lista de archivos en la carpeta de origen de Google Drive
results = drive_service.files().list(q=f"'{source_folder_id}' in parents", fields='files(id, name)').execute()
files = results.get('files', [])

# Iterar sobre los archivos, descargarlos y luego cargarlos en la carpeta de destino
for file in files:
    file_name = file['name']
    download_path = os.path.join(temp_dir, file_name)

    # Descargar el archivo de Google Drive
    descargar_archivo(file['id'], download_path)

    # Subir el archivo a la carpeta de destino
    subir_archivo_a_carpeta(download_path, file_name)

print("Archivos descargados y sobrescritos en Google Drive.")




Descargado 100%.
Descargado 100%.
Archivo actualizado en Google Drive: InvoicePurchases12312016_Clean.csv
Archivo actualizado en Google Drive: 2017PurchasePricesDec_Clean.csv
Descargando archivo 'Carpeta de origen/2017PurchasePricesDec_Clean.csv' desde la carpeta 'Carpeta de origen'.
Datos del archivo de origen '2017PurchasePricesDec_Clean.csv':
Descargando archivo 'Carpeta de origen/InvoicePurchases12312016_Clean.csv' desde la carpeta 'Carpeta de origen'.
Datos del archivo de origen 'InvoicePurchases12312016_Clean.csv':
Descargando archivo 'Carpeta de comparación/comparison_2017PurchasePricesDec_Clean.csv' desde la carpeta 'Carpeta de comparación'.
Datos del archivo de comparación '2017PurchasePricesDec_Clean.csv':
Realizando comparación entre los datos de '2017PurchasePricesDec_Clean.csv' en la carpeta de origen y en la carpeta de comparación.
Se encontraron filas nuevas en el archivo '2017PurchasePricesDec_Clean.csv':
Archivo 2017PurchasePricesDec_Clean.csv subido a Google Drive en 